In [23]:
import os
import torch
import imageio
import argparse
import numpy as np
from glob import glob
from tqdm import tqdm, trange
import tensorflow as tf2
import tensorflow.compat.v1 as tf
from tensorflow import io
from os.path import join, exists

In [21]:
def read_bytes(path):
    '''Read bytes for OpenSeg model running.'''

    with io.gfile.GFile(path, 'rb') as f:
        file_bytes = f.read()
    return file_bytes

In [4]:
#load openseg model
openseg_model = tf2.saved_model.load("C:/Users/aorhu/Masaüstü/AT3DCV/repo/openseg_model",tags=[tf.saved_model.tag_constants.SERVING],)

In [9]:
split = 'train'
n_split_points = 20000
num_rand_file_per_scene = 5


data_dir = "D:/AT3DCV_Data/Preprocessed_OpenScene/data"
data_root = join(data_dir, 'scannet_3d')
data_root_2d = join(data_dir,'scannet_2d')

data_paths = sorted(glob(join(data_root, split, '*.pth')))

out_dir = "D:/AT3DCV_Data/Preprocessed_OpenScene/data/features_test"

In [13]:
data_paths[0]

'D:/AT3DCV_Data/Preprocessed_OpenScene/data\\scannet_3d\\train\\scene0000_00_vh_clean_2.pth'

In [14]:
scene_id = data_paths[0].split('/')[-1].split('\\')[-1].split('_vh')[0]

In [15]:
scene_id

'scene0000_00'

In [41]:
scene = join(data_root_2d, scene_id)
img_dirs = sorted(glob(join(scene, 'color/*')), key=lambda x: int(os.path.basename(x)[:-4]))
num_img = len(img_dirs)

In [43]:
img_dirs[0]

'D:/AT3DCV_Data/Preprocessed_OpenScene/data\\scannet_2d\\scene0000_00\\color\\0.jpg'

In [36]:
text_emb = tf.zeros([1, 1, 768])

In [37]:
regional_pool = True #by default

In [39]:
img_features = []
for img_dir in tqdm(img_dirs):
    np_image_string = read_bytes(img_dir)
    results = openseg_model.signatures['serving_default'](
                inp_image_bytes = tf.convert_to_tensor(np_image_string),
                inp_text_emb = text_emb)
    
    img_info = results['image_info']
    crop_sz = [int(img_info[0, 0] * img_info[2, 0]), int(img_info[0, 1] * img_info[2, 1])]
    
    if regional_pool:
        image_embedding_feat = results['ppixel_ave_feat'][:, :crop_sz[0], :crop_sz[1]]
    else:
        image_embedding_feat = results['image_embedding_feat'][:, :crop_sz[0], :crop_sz[1]]

    feat_2d = tf.cast(tf.image.resize_nearest_neighbor(
            image_embedding_feat, [240,320], align_corners=True)[0], dtype=tf.float16).numpy()
    feat_2d = torch.from_numpy(feat_2d).permute(2, 0, 1)
    break

  0%|                                                                                          | 0/279 [00:05<?, ?it/s]


In [30]:
results.keys()

dict_keys(['region_probs_', 'text_embedding', 'segm_proposal_feats', 'image_embedding_feat', 'pixel_pred_confidence', 'segm_confidence', 'segm_prediction', 'images', 'region_embeddings', 'region_probs', 'ppixel_ave_feat', 'ppixel_ave_feat_confidence', 'segm_confidence_rw', 'image', 'segm_prediction_rw', 'image_info', 'ppixel_ave_feat_pred', 'pixel_prediction', 'segm_proposal', 'region_logits'])

In [ ]:
image_embedding_feat

In [40]:
feat_2d.shape

torch.Size([768, 240, 320])

In [42]:
feat_2d

tensor([[[ 3.0078e-01,  2.4695e-01,  2.4695e-01,  ...,  2.5781e-01,
           2.5781e-01,  2.5781e-01],
         [ 1.6809e-01,  2.2083e-01,  2.2083e-01,  ...,  2.5781e-01,
           2.5781e-01,  2.5781e-01],
         [ 1.4209e-01,  9.3323e-02,  1.7212e-01,  ...,  2.5781e-01,
           2.5781e-01,  2.5781e-01],
         ...,
         [ 9.4666e-02,  5.4077e-02,  5.4077e-02,  ...,  3.4973e-02,
           3.4973e-02,  3.4973e-02],
         [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
           0.0000e+00,  0.0000e+00],
         [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
           0.0000e+00,  0.0000e+00]],

        [[ 2.9980e-01,  1.9580e-01,  1.9580e-01,  ...,  1.4075e-01,
           1.4075e-01,  1.4075e-01],
         [ 1.4307e-01,  1.4001e-01,  1.4001e-01,  ...,  1.4075e-01,
           1.4075e-01,  1.4075e-01],
         [ 8.7158e-02,  7.1838e-02,  1.2469e-01,  ...,  1.4075e-01,
           1.4075e-01,  1.4075e-01],
         ...,
         [ 8.6121e-02,  4